In [6]:
%pip install sshtunnel psycopg2-binary sqlalchemy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

In [9]:
load_dotenv(dotenv_path="D:\BRC-KingsGenomics\Scripts\prada-scripts\login.env")

True

In [10]:
SSH_HOST = os.getenv('SSH_HOST')
SSH_PORT = int(os.getenv('SSH_PORT', 22))
SSH_USER = os.getenv('SSH_USER')
SSH_PASSWORD = os.getenv('SSH_PASSWORD')

DB_HOST = os.getenv('DB_HOST')
DB_PORT = int(os.getenv('DB_PORT', 5432))
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')


In [11]:
# Create the SSH tunnel
with SSHTunnelForwarder(
    (SSH_HOST, SSH_PORT),
    ssh_username=SSH_USER,
    ssh_password=SSH_PASSWORD,
    remote_bind_address=(DB_HOST, DB_PORT)) as tunnel:

    conn = psycopg2.connect(
        host='127.0.0.1',
        port=tunnel.local_bind_port,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME)


In [12]:
# PGx gene list

def fetch_prioritised_gene_regions(conn, gene_list):
    query = f"""
    SELECT chr, bp1, bp2, gene_name
    FROM prada.gencode_gene
    WHERE gene_name IN ({','.join(["%s"] * len(gene_list))})
    """
    return pd.read_sql(query, conn, params=gene_list)


In [13]:

file_path = "D:\BRC-KingsGenomics\PRADA\PGx Adaptive Sampling Gene List (Draft)(Adaptive).csv"
gene_list_df = pd.read_csv(file_path)


gene_list_df.head(), gene_list_df.columns.tolist()

(    Gene CPIC PharmGKB  DPWG  Twist  PharmVar  CMRG  PharmCAT
 0  ABCB1    C    3 VIP   NaN    1.0       NaN   1.0       NaN
 1  ABCC4    D        3   NaN    NaN       NaN   NaN       NaN
 2  ABCG2    A       1A   1.0    1.0       NaN   1.0       1.0
 3   ABL2    B      NaN   NaN    NaN       NaN   1.0       NaN
 4    ACE    D       2A   NaN    NaN       NaN   NaN       NaN,
 ['Gene', 'CPIC', 'PharmGKB', 'DPWG', 'Twist', 'PharmVar', 'CMRG', 'PharmCAT'])

In [14]:
pgx_genes = gene_list_df['Gene'].dropna().str.upper().unique().tolist()

def fetch_gene_regions_for_pgx(conn, gene_list):
    sql = f"""
        SELECT chr, bp1 AS start, bp2 AS end, gene_name
        FROM prada.gencode_gene
        WHERE gene_name IN ({','.join(['%s'] * len(gene_list))})
    """
    return pd.read_sql(sql, conn, params=gene_list)

In [15]:
def write_bed(df, output_path):
    bed = df[['chr', 'start', 'end']].copy()
    bed = bed.sort_values(by=['chr', 'start'])
    bed.to_csv(output_path, sep="\t", header=False, index=False)
    print(f"BED file written to {output_path}")

In [16]:

def fetch_and_write_bed_via_tunnel(gene_list, output_path):
    with SSHTunnelForwarder(
        (SSH_HOST, SSH_PORT),
        ssh_username=SSH_USER,
        ssh_password=SSH_PASSWORD,
        remote_bind_address=(DB_HOST, DB_PORT)
    ) as tunnel:

        with psycopg2.connect(
            host="127.0.0.1",
            port=tunnel.local_bind_port,
            user=DB_USER,
            password=DB_PASS,
            database=DB_NAME
        ) as conn:

            sql = f"""
                SELECT chr, bp1 AS start, bp2 AS end, gene_name
                FROM prada.gencode_gene
                WHERE gene_name IN ({','.join(['%s'] * len(gene_list))})
            """
            df = pd.read_sql(sql, conn, params=gene_list)

            # Write BED file
            bed = df[['chr', 'start', 'end']].sort_values(by=['chr', 'start'])
            bed.to_csv(output_path, sep='\t', header=False, index=False)
            print(f"BED file written to {output_path}")


In [18]:
# Get regions and write to BED
    # pgx_regions = fetch_gene_regions_for_pgx(conn, pgx_genes)
    # write_bed(pgx_regions, "pgx_prioritised_genes.bed")

fetch_and_write_bed_via_tunnel(pgx_genes, "D:\BRC-KingsGenomics\PRADA\pgx_prioritised_genes2.bed")


BED file written to D:\BRC-KingsGenomics\PRADA\pgx_prioritised_genes2.bed


In [19]:
def calculate_coverage(bed_df):
    bed_df = bed_df.sort_values(by=["chr", "start"])

    total = 0
    current_chr = None
    current_start = current_end = None

    for _, row in bed_df.iterrows():
        chr_, start, end = row["chr"], row["start"], row["end"]

        if chr_ != current_chr:
            if current_chr is not None:
                total += current_end - current_start
            current_chr, current_start, current_end = chr_, start, end
        else:
            if start > current_end:
                total += current_end - current_start
                current_start, current_end = start, end
            else:
                current_end = max(current_end, end)

    if current_chr is not None:
        total += current_end - current_start

    return total


In [20]:
bed_path = "D:\BRC-KingsGenomics\PRADA\pgx_prioritised_genes2.bed"
bed_df = pd.read_csv(bed_path, sep="\t", header=None, names=["chr", "start", "end"])

# primary chromosomes
primary_chromosomes = [f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY"]
primary_bed_df = bed_df[bed_df["chr"].isin(primary_chromosomes)].copy()

primary_bed_df.head()


,chr,start,end
62,chr1,11785723,11806455
63,chr1,20589086,20618903
64,chr1,28812170,28871267
65,chr1,78303884,78540701
66,chr1,97077743,97995000


In [ ]:
# # Add Gene & Update BED
# def add_gene_to_backbone(conn, gene_name, padding=1000, source="custom"):
    # query = """
    #     SELECT chr, bp1 - %s AS start, bp2 + %s AS end, gene_name
    #     FROM prada.gencode_gene
    #     WHERE gene_name = %s
    # """
#     gene_info = pd.read_sql(query, conn, params=[padding, padding, gene_name])
    
#     if gene_info.empty:
#         raise ValueError(f"Gene '{gene_name}' not found.")
    
#     row = gene_info.iloc[0]
#     insert_sql = """
#         INSERT INTO pgx_backbone_genes (gene_name, chr, start, end, source)
#         VALUES (%s, %s, %s, %s, %s)
#         ON CONFLICT (gene_name) DO NOTHING;
#     """
#     with conn.cursor() as cur:
#         cur.execute(insert_sql, (gene_name, row["chr"], row["start"], row["end"], source))
#         conn.commit()

# # BED Export Function
# def export_backbone_bed(conn, output_path):
#     df = pd.read_sql("SELECT chr, start, end FROM pgx_backbone_genes", conn)
#     df.sort_values(by=["chr", "start"]).to_csv(output_path, sep="\t", header=False, index=False)


# # SAM coverage
# import pysam

# def region_coverage(bam_path, chrom, start, end):
#     bam = pysam.AlignmentFile(bam_path, "rb")
#     coverage = bam.count_coverage(chrom, start, end)
#     return sum(map(sum, coverage)) / (end - start)
